__Obiettivo__ 

Forzare la classificazione binaria secondo un approccio non-supervisionato, adeguando qualche algoritmo di similarità del testo.

In [1]:
import pandas

from nltk.corpus import stopwords

set_stopwords = set(stopwords.words("english"))

In [515]:
import json

with open("../../json/topics/topics.json", "r") as file:
    dict_topics = json.load(file)

df = pandas.read_json("../../json/dataset/dataset_preprocessed.json")

In [516]:
df.head(10)

,DOI,Title,Keywords,Abstract,Introduction,Text
2,10.1007/3-540-44595-1_3,intelligent tutor web based chess course,"[textbooks, chess, teachers, tutor, tutoring]",web based intelligent tutoring system try fill...,course common way organize teaching high level...,web based intelligent tutoring system try fill...
5,10.3233/icg-1998-21203,learning play chess using temporal difference,"[td, knightcap, algorithm, internet, chess]",paper present tdleaf variation algorithm enabl...,temporal difference learning first introduced ...,paper present tdleaf variation algorithm enabl...
7,10.1016/s0020-0255(99)00093-6,temporal dierence learning heuristic search ga...,"[backgammon, championship, algorithmic, td, ch...",temporal dierence learning natural method rein...,central heuristic search whether single agent ...,temporal dierence learning natural method rein...
8,10.1006/ijhc.1999.0338,incremental acquisition search knowledge,"[hampered, intractable, expert, intelligence, ...",expert system nition address particular domain...,early day arti cial intelligence strong emphas...,expert system nition address particular domain...
9,10.1016/s0020-0255(99)00097-3,risk management game tree pruning,"[century, strategy, search, algorithm, algorit...",half century since minimax rst suggested strat...,standard approach game tree search use improve...,half century since minimax rst suggested strat...
11,10.1006/jpdc.1999.1600,aphid asynchronous parallel game tree search,"[search, synchronous, asynchronous, efficient,...",parallel game tree search approach use synchro...,making computer play game skillful manner comp...,parallel game tree search approach use synchro...
12,10.1109/mc.2017.177,competition controversy computer chess,None,,,
13,10.1017/s0140525x01003922,magical number short term memory reconsiderati...,"[memory, overload, evidence, seven, rehearsal]",summarized evidence people remember seven chun...,one central contribution cognitive psychology ...,summarized evidence people remember seven chun...
17,10.1037//0033-295x.107.3.578,expert adaptation representative task demand a...,"[rejected, experts, critique, refutation, misr...",theoretical framework long term working memory...,example chess player could play chess blindfol...,theoretical framework long term working memory...
18,10.1006/game.1999.0763,chess like game dominance solvable two step,"[solved, elimination, rounds, games, competitive]",show strictly competitive nite game perfect in...,probably rst contribution pure game theory,show strictly competitive nite game perfect in...


In [517]:
list_topics = [topic for topic in dict_topics.keys()]

In [518]:
from typing import List, Dict
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def preprocessed_topics(topics: str) -> List[str]:
    _list = []
    lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+")

    for topic in topics:
        tokens = tokenizer.tokenize(topic.lower())

        tokens = [token for token in tokens if len(token) > 2]
        tokens = [token for token in tokens if token not in set_stopwords]

        lemma =  [lemmatizer.lemmatize(token) for token in tokens]
        
        _list.append(" ".join(lemma))

    return _list

def take_kwords_from_df(feature: str) -> List[str]:
    _list = []
    list_kwords: List[str]  = list(df[feature])

    for item in list_kwords:
        if item is not None:
            _list.append(" ".join(item))

    return _list

def take_feature_from_df(feature: str) -> Dict[str, str]:
    _dict = {}
    _df = df[["DOI", feature]]

    for _, value in _df.iterrows():
        if len(value[feature]) > 0:
            _dict[value["DOI"]] = value[feature]
            
    return _dict

topics = preprocessed_topics(list_topics)

kwords = take_kwords_from_df("Keywords")
dict_entities = take_feature_from_df("Abstract")

In [534]:
import json

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

_dict = {}
for key, value in dict_entities.items():
    dict_similarities = {}

    for topic in topics:
        corpus = [value, topic]
        vectorized_corpus = vectorizer.fit_transform(corpus)

        vectorized_text = vectorized_corpus[0]
        vectorized_topic = vectorized_corpus[1]

        similarity = cosine_similarity(vectorized_text, vectorized_topic)

        dict_similarities[topic] = similarity.max()

    _dict[key] = dict_similarities

In [535]:
with open("../../json/classification/classification_multi_binary.json", "w") as file:
    json.dump(_dict, file, indent=3)